<a href="https://colab.research.google.com/github/nandhukumar86/CapstonePneumoniaDetection/blob/main/Vani/Capstoneyolomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#MASK rcnn-ojcect detection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
TrainDir = "/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_train/"
TestDir='/content/drive/MyDrive/Content/DriveAIML/capstone/jpg_reshaped_test/'
sublabel='/content/drive/MyDrive/Content/DriveAIML/capstone/stage_2_sample_submission.csv'
basepath = "/content/drive/MyDrive/Content/DriveAIML/capstone/"

In [6]:
import os, cv2,copy, gc
import pickle
import glob2 as glob
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers, Model, losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation, BatchNormalization

In [7]:
filename =  '/content/drive/MyDrive/Content/DriveAIML/capstone/RezisedLabel.pickle'
with open(filename, "rb") as input_file:
  y_train = pickle.load(input_file)

In [8]:
trainimg = glob.glob(TrainDir+'*.jpg') #Getting all images in this folder
testimg=glob.glob(TestDir+"*.jpg") #

In [9]:
 len(trainimg)

11740

In [10]:
y_train.head(5)


,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,1024,1024,256,256,66.0,38.0,54.0,95.0


In [11]:
y_train.size

423178

In [12]:
class_info_df = pd.read_csv(basepath+'stage_2_detailed_class_info.csv')
sub_info_df=pd.read_csv(sublabel)#test label or submission 

In [13]:
## Merging the two datasets
#In class detailed info dataset are given the detailed information about 
#the type of positive or negative class associated with a certain patient.
#In y_train labels dataset are given the patient ID and the window (x min, y min, width and height of the) 
#containing evidence of pneumonia
train_class_df = y_train.merge(class_info_df, left_on= ('patientId'), right_on=('patientId'), how='inner').drop_duplicates()
train_class_df.sample(5)




,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto,class
19760,9a62135f-b84d-49c0-bfe6-359599842531,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal
4927,38b0701d-e90f-4352-8b40-1952544cb824,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal
20016,9c61da04-bb7c-4e49-b9d2-04c1f59ad7e3,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal
23347,b01d24dd-ff2e-4b9b-99bc-29023fc5b3ec,502.0,191.0,161.0,279.0,1,1024,1024,256,256,126.0,48.0,41.0,70.0,Lung Opacity
10778,5e9a4de3-dc31-4395-a1fa-4f5209023cbf,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal


In [14]:
#reading image 500 samples traindir
sampNum = 500 # Number of samples to import
X = [] # Empty to read training images
x1=[]
print(type(x1))
print("Before sampling\n")
print("Total training unique patient ids: ",train_class_df['patientId'].nunique())
print("Number of images in train set:", len(os.listdir(TrainDir)))
print("\n")
print("Total training labels {}" .format(train_class_df.shape))
#display(distTab(train_class_df, "Target")) # Dsitribution of target before sampling

print("\n\nAfter sampling\n")
Y = train_class_df.sample(n=sampNum, random_state=2) # Sampling
Y.reset_index(drop=True, inplace=True)
print("Total sampled training labels {}" .format(Y.shape))
#display(distTab(Y.sample(n=sampNum), "Target"))  # Dsitribution of target after sampling

for indx in Y.patientId:
    
    try:
       #print(indx)
       filepath=TrainDir+indx+'.jpg'
       image = cv2.imread(filepath)
       X.append(indx)
       x1.append(image)
      # print(filepath)
    except: 
        continue
zipbObj = zip(X, x1)        
 

X=dict(zipbObj)#takes zip object
print("\n\nTotal sampled training images {}" .format(len(X)))
Y = Y[Y.patientId.isin(X.keys())]
print("Total training labels {}" .format(Y.shape))

<class 'list'>
Before sampling

Total training unique patient ids:  26684
Number of images in train set: 11740


Total training labels (30227, 15)


After sampling

Total sampled training labels (500, 15)


Total sampled training images 500
Total training labels (500, 15)


In [15]:
#reading image 100 samples test dir
sampNum = 10 # Number of samples to import
Xtest = [] # Empty to read training images
x1test=[] 
#print("Total training unique patient ids: ",train_class_df['patientId'].nunique())
print("Total training unique patient ids: ",sub_info_df['patientId'].nunique())
print("Number of images in test set:", len(os.listdir(TestDir)))
 
print("\n")
print("Total training labels {}" .format(sub_info_df.shape))
#print("Total training labels {}" .format(train_class_df.shape))
#Ytest = train_class_df.sample(n=sampNum, random_state=2) # Sampling
Ytest = sub_info_df.sample(n=sampNum, random_state=2) # Sampling
Ytest.reset_index(drop=True, inplace=True)
print("Total sampled testing labels {}" .format(Ytest.shape))
#display(distTab(Y.sample(n=sampNum), "Target"))  # Dsitribution of target after sampling

for indx1 in Ytest.patientId:
    
    try:
      # print(indx1)
       filepath1=TestDir+indx1+'.jpg'
       image1 = cv2.imread(filepath1)
       Xtest.append(indx1)
       x1test.append(image1)
      # print(filepath1)
    except: 
        continue
zipbObj1 = zip(Xtest, x1test)        
 

Xtest=dict(zipbObj1)#takes list object
#print(Xtest)
print("\n\nTotal sampled test images {}" .format(len(Xtest)))
Ytest = Ytest[Ytest.patientId.isin(Xtest.keys())]
print("Total test labels {}" .format(Ytest.shape))

Total training unique patient ids:  3000
Number of images in test set: 2871


Total training labels (3000, 2)
Total sampled testing labels (10, 2)


Total sampled test images 10
Total test labels (10, 2)


In [16]:
# Shape of images
print(len(X))
print(len(Xtest))
print(len(Y))
print(len(Ytest))


500
10
500
10


In [17]:
Xdf = pd.DataFrame({'patientId':X.keys(),
                   'imageArray':X.values()})
dattrain = Y.merge(Xdf, on='patientId', how='left')

#test
Xtestdf = pd.DataFrame({'patientId':Xtest.keys(),
                   'imageArray':Xtest.values()})
dattest = Ytest.merge(Xtestdf, on='patientId', how='left')

In [18]:
dattrain.head(10)

,patientId,x,y,width,height,Target,X,Y,Xo,Yo,xo,yo,widtho,heighto,class,imageArray
0,e4dd25aa-7839-4e0e-a984-c7cb1b31d2c7,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,"[[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], ..."
1,5c389c6d-2abc-43a9-91e7-847f85bf6ff1,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
2,324f9ed1-09e0-483e-9bdd-d139e77acb85,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,None
3,4ac6590a-ecdb-47b7-92f3-cfc6fa405955,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
4,7213e146-4efd-40e6-ab88-a9e7a9c8c25c,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
5,229e6aef-1467-49a2-9158-49fd8acea0cf,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,None
6,8c66c1c4-02c3-47ff-b2af-748370dd8b7e,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,None
7,ef08ab22-8cd0-4f63-af8a-c839a94916db,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
8,ea3347c3-e4eb-4c00-927b-176ddd296f72,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,Normal,"[[[255, 255, 255], [253, 253, 253], [254, 254,..."
9,1ed2052a-c208-40ca-8ccc-3378aec6f929,NaN,NaN,NaN,NaN,0,1024,1024,256,256,NaN,NaN,NaN,NaN,No Lung Opacity / Not Normal,None


In [19]:
 dattest.head(10)

,patientId,PredictionString,imageArray
0,1cb71dda-47f2-44e0-ad42-2737ec3f43b6,0.5 0 0 100 100,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
1,0f9f2955-cf07-40e3-9624-6d84654c9fb9,0.5 0 0 100 100,"[[[4, 4, 4], [6, 6, 6], [7, 7, 7], [7, 7, 7], ..."
2,03675081-3f27-4e22-a1dd-6cccd8c4cc92,0.5 0 0 100 100,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
3,30fe50ee-4108-42fe-8dfd-27b1e61dccf0,0.5 0 0 100 100,"[[[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], ..."
4,1cb7194e-f761-41b7-8e6f-d5abdbfb270c,0.5 0 0 100 100,"[[[217, 217, 217], [213, 213, 213], [211, 211,..."
5,1e585120-583c-4d50-be9b-8ffda0f933a9,0.5 0 0 100 100,"[[[130, 130, 130], [124, 124, 124], [117, 117,..."
6,2d17ce84-86f9-4153-8d91-3a75ffe303cc,0.5 0 0 100 100,"[[[70, 70, 70], [28, 28, 28], [11, 11, 11], [9..."
7,10aea5ae-b0be-48e1-87cd-477c15a85772,0.5 0 0 100 100,"[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [1, 1, 1], ..."
8,048a59f8-9022-4598-8424-f8b0250a2e2a,0.5 0 0 100 100,"[[[3, 3, 3], [3, 3, 3], [3, 3, 3], [3, 3, 3], ..."
9,c04e3eeb-80ec-46f1-81e4-4186ece11b3e,0.5 0 0 100 100,"[[[2, 2, 2], [2, 2, 2], [2, 2, 2], [2, 2, 2], ..."


In [ ]:
#
#test_class_df = y_train.merge(sub_info_df, left_on= ('patientId'), right_on=('patientId'), how='inner').drop_duplicates()
#train_class_df.describe

MSK RCC model


In [20]:
#download fro  gir
!git clone https://www.github.com/matterport/Mask_RCNN.git
os.chdir('Mask_RCNN')
#!python setup.py -q install





Cloning into 'Mask_RCNN'...
remote: Enumerating objects: 956, done.
remote: Total 956 (delta 0), reused 0 (delta 0), pack-reused 956
Receiving objects: 100% (956/956), 125.23 MiB | 33.24 MiB/s, done.
Resolving deltas: 100% (562/562), done.


In [28]:
# Import Mask RCNN in root folder
import sys,os
sys.path.append(os.path.join(basepath, 'Mask_RCNN'))  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
#import mrcnn.model as modellib
from mrcnn import visualize
#from mrcnn.model import log

AttributeError: ignored

In [25]:
#download weights
### Download COCO pre-trained weights
!wget --quiet https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!ls -lh mask_rcnn_coco.h5

COCO_WEIGHTS_PATH = "/kaggle/input/mask-rcnn-coco/mask_rcnn_coco.h5"

-rw-r--r-- 1 root root 246M Nov 26  2017 mask_rcnn_coco.h5


In [26]:
hyper_paramters_comb={
    'backbone':['resnet50'],
    'learning_rate':[0.005],
    'batch_size':[8],
    'epochs':[10],
    'det_min_conf':[0.9],
    'det_nms_th':[0.8],
    'rpn_nms_th':[0.7],
    'steps_per_epoch':[135],
    'layers': ['heads']
}

hpc=pd.DataFrame(hyper_paramters_comb)

hpc['learning_rate'] = hpc['learning_rate'].astype(np.float32)
hpc['det_min_conf'] = hpc['det_min_conf'].astype(np.float32)
hpc['det_nms_th'] = hpc['det_nms_th'].astype(np.float32)
hpc['rpn_nms_th'] = hpc['rpn_nms_th'].astype(np.float32)

hpc.head()

,backbone,learning_rate,batch_size,epochs,det_min_conf,det_nms_th,rpn_nms_th,steps_per_epoch,layers
0,resnet50,0.005,8,10,0.9,0.8,0.7,135,heads


In [27]:
model = modellib.MaskRCNN(mode='training', config=config, model_dir=basepath)

# Exclude the last layers because they require a matching number of classes
model.load_weights(COCO_WEIGHTS_PATH, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

NameError: ignored